In [ ]:
from ultralytics import YOLO
import cv2
import os
import csv
import re

# ====== Cấu hình Thư mục ======
image_dir = r"C:/Users/HP/Videos/Captures/anh_phat_hien/data"
save_img_dir = r"C:/Users/HP/Videos/Captures/kq_cua_test"
CSV_OUTPUT_PATH = r"C:/Users/HP/Videos/Captures/vehicle_count.csv"

os.makedirs(save_img_dir, exist_ok=True)

# ====== Các lớp cần detect ======
TARGET_CLASSES = ['car', 'motorcycle', 'truck']

# ====== Load mô hình YOLOv8 ======
model = YOLO("yolov8x.pt")

# ====== Hàm trích số từ tên file ======
def extract_number(filename):
    match = re.search(r'\d+', filename)
    return int(match.group()) if match else -1

# ====== Danh sách ảnh ======
image_files = sorted(
    [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))],
    key=extract_number
)

print(f"🔍 Bắt đầu xử lý {len(image_files)} ảnh và ghi vào CSV: {CSV_OUTPUT_PATH}")

# ====== Chuẩn bị file CSV ======
csv_header = [
    'Image_Index', 'Original_Filename', 'Total_Vehicles',
    'Motorcycles', 'Cars', 'Trucks',
    'Truck_Percentage'
]

with open(CSV_OUTPUT_PATH, 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(csv_header)

    for idx, filename in enumerate(image_files, start=1):
        image_path = os.path.join(image_dir, filename)
        frame = cv2.imread(image_path)

        if frame is None:
            print(f" Không đọc được ảnh: {filename}")
            continue

        results = model.predict(source=frame, imgsz=1536, conf=0.25, iou=0.5, augment=True, verbose=False)
        boxes = results[0].boxes

        # Đếm từng loại xe
        count_motorcycle = 0
        count_car = 0
        count_truck = 0

        for box in boxes:
            cls_id = int(box.cls[0])
            cls_name = model.names[cls_id]

            if cls_name in TARGET_CLASSES:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, cls_name, (x1, max(20, y1 - 10)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, cv2.LINE_AA)

                if cls_name == 'motorcycle':
                    count_motorcycle += 1
                elif cls_name == 'car':
                    count_car += 1
                elif cls_name == 'truck':
                    count_truck += 1

        total_vehicles = count_motorcycle + count_car + count_truck
        #  Truck_Percentage = (car + truck) / total
        truck_percentage = round(((count_car + count_truck) / total_vehicles) * 100, 2) if total_vehicles > 0 else 0.0

        # Ghi kết quả
        match = re.search(r'\d+', filename)
        new_name = match.group(0).zfill(5) if match else f"{idx:05d}"

        csv_row = [
            new_name, filename, total_vehicles,
            count_motorcycle, count_car, count_truck,
            truck_percentage
        ]
        csv_writer.writerow(csv_row)

        output_img_path = os.path.join(save_img_dir, new_name + ".jpg")
        cv2.imwrite(output_img_path, frame)

        print(f"{filename} → Tổng: {total_vehicles} | Xe máy: {count_motorcycle} | Ô tô: {count_car} | Xe tải: {count_truck} | Tải+Ôtô %: {truck_percentage}%")

print("\n Hoàn tất! Dữ liệu đã được ghi vào CSV và ảnh kết quả được lưu.")

🔍 Bắt đầu xử lý 2640 ảnh và ghi vào CSV: C:/Users/HP/Videos/Captures/vehicle_count.csv
001.jpg → Tổng: 15 | Xe máy: 7 | Ô tô: 5 | Xe tải: 3 | Tải+Ôtô %: 53.33%


KeyboardInterrupt: 